In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hacklive-3-guided-hackathon-nlp/Test.csv
/kaggle/input/hacklive-3-guided-hackathon-nlp/SampleSubmission.csv
/kaggle/input/hacklive-3-guided-hackathon-nlp/Tags.csv
/kaggle/input/hacklive-3-guided-hackathon-nlp/Train.csv


In [2]:
# !wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz


In [3]:
# # !mkdir scibert_scivocab_uncased
# !tar -xvf ./scibert_scivocab_uncased.tar.gz
# ! mv scibert_scivocab_uncased/bert_config.json scibert_scivocab_uncased/config.json
# ! mv scibert_scivocab_uncased/bert_model.ckpt.data-00000-of-00001 scibert_scivocab_uncased/tf_model.h5
# !ls scibert_scivocab_uncased


In [4]:
!pip install -q transformers==2.11.0
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz
!tar -xvf ./scibert_scivocab_uncased.tar.gz
os.environ["WANDB_API_KEY"] = "0" ## to silence warning
!transformers-cli convert --model_type bert \
  --tf_checkpoint './scibert_scivocab_uncased/bert_model.ckpt' \
  --config './scibert_scivocab_uncased/bert_config.json' \
  --pytorch_dump_output './scibert_scivocab_uncased/pytorch_model.bin'

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

allennlp 1.1.0 requires transformers<3.1,>=3.0, but you'll have transformers 2.11.0 which is incompatible.
--2020-10-23 12:34:38--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.181.208
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.181.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1216161420 (1.1G) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar.gz’

scibert_scivocab_un 100%[===================>]   1.13G  37.8MB/s    in 29s     

2020-10-23 12:35:08 (39.7 MB/s) - ‘scibert_scivocab_unca

In [5]:

import tensorflow as tf
import transformers
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
import pickle as pkl
import gc
import logging
import warnings
import time
import sys

In [6]:
bert_model_name = './scibert_scivocab_uncased'

config = transformers.BertConfig.from_json_file('./scibert_scivocab_uncased/bert_config.json')

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)


In [7]:

max_length = 512
batch_size = 32
epochs = 5
df = pd.read_csv('/kaggle/input/hacklive-3-guided-hackathon-nlp/Train.csv')

X = df.iloc[:, 1: 6]
y = df.iloc[:, 6: 6 + 25]

LABELS = y.columns


train_X, test_X, train_y, test_y = train_test_split(X, y, shuffle=True, test_size=.2)
train_X.shape, train_y.shape, test_X.shape, test_y.shape


del train_X, train_y



In [8]:


X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)
test_X = test_X.reset_index(drop=True)


test_y = test_y.values
y = y.values



In [9]:


class BertSemanticDataGenerator(tf.keras.utils.Sequence):

    def __init__(
        self,
        sentences,
        depts,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentences = sentences
        self.depts = depts
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
#         self.tokenizer = transformers.BertTokenizer.from_pretrained(
#             "bert-base-uncased", do_lower_case=True, max_length=2048
#             )
        self.tokenizer = transformers.BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
        self.indexes = np.arange(len(self.sentences))
        self.on_epoch_end()

    def __len__(self):
        return len(self.sentences) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentences = self.sentences[indexes]

        encoded = self.tokenizer.batch_encode_plus(
            sentences.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )


        input_ids = np.array(encoded["input_ids"], dtype="int32")

        dept_ids = np.array(self.depts[indexes], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, dept_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, dept_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


In [10]:

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def macro_double_soft_f1(y, y_hat):
    
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    tn = tf.reduce_sum((1 - y_hat) * (1 - y), axis=0)
    soft_f1_class1 = 2*tp / (2*tp + fn + fp + 1e-16)
    soft_f1_class0 = 2*tn / (2*tn + fn + fp + 1e-16)
    cost_class1 = 1 - soft_f1_class1 # reduce 1 - soft-f1_class1 in order to increase soft-f1 on class 1
    cost_class0 = 1 - soft_f1_class0 # reduce 1 - soft-f1_class0 in order to increase soft-f1 on class 0
    cost = 0.5 * (cost_class1 + cost_class0) # take into account both class 1 and class 0
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost



In [11]:

input_ids = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="input_ids"
)

input_departments = tf.keras.Input(
    shape = (4,), dtype=tf.float32, name='input_depts'
)

# Attention masks indicates to the model which tokens should be attended to.
attention_masks = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="attention_masks"
)

# Token type ids are binary masks identifying different sequences in the model.
token_type_ids = tf.keras.layers.Input(
    shape=(max_length,), dtype=tf.int32, name="token_type_ids"
)

# Loading pretrained BERT model.
# bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
bert_model = transformers.TFBertModel.from_pretrained(bert_model_name, from_pt=True, config = config)
# Freeze the BERT model to reuse the pretrained features without modifying them.
bert_model.trainable = False

sequence_output, pooled_output = bert_model(
    input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
)

# Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
bi_lstm = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(64, return_sequences=True)
)(sequence_output)

# Applying hybrid pooling approach to bi_lstm sequence output.
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
concat = tf.keras.layers.concatenate([avg_pool, max_pool, input_departments])
dropout = tf.keras.layers.Dropout(0.3)(concat)
output = tf.keras.layers.Dense(25, activation="sigmoid")(dropout)
model = tf.keras.models.Model(
    inputs=[input_ids, input_departments, attention_masks, token_type_ids], outputs=output
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=["acc", f1_m],
)

model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 512)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 512, 768), ( 109918464   input_ids[0][0]                  
                                                                 attention_masks[0][0] 

In [12]:
# debug

In [13]:
val_data = BertSemanticDataGenerator(
    test_X['ABSTRACT'].values.astype("str"),
    test_X.iloc[:,1:6].values.astype("int32"),
    test_y,
    batch_size=batch_size,
    shuffle=False,
)


data = BertSemanticDataGenerator(
    X['ABSTRACT'].values.astype("str"),
    X.iloc[:,1:6].values.astype("int32"),
    y,
    batch_size=batch_size,
    shuffle=True,
)




print("training model")
history = model.fit(
    data,
    validation_data=val_data,
    shuffle=True,
    epochs=epochs,
)

K.clear_session()
gc.collect()
print("sleeping")
# time.sleep(60)


training model
Epoch 1/5
437/437 [==============================] - 427s 977ms/step - loss: 0.1308 - acc: 0.4322 - f1_m: 0.3825 - val_loss: 0.0825 - val_acc: 0.6135 - val_f1_m: 0.6540
Epoch 2/5
437/437 [==============================] - 422s 965ms/step - loss: 0.0836 - acc: 0.6037 - f1_m: 0.6367 - val_loss: 0.0670 - val_acc: 0.6480 - val_f1_m: 0.7173
Epoch 3/5
437/437 [==============================] - 422s 967ms/step - loss: 0.0733 - acc: 0.6427 - f1_m: 0.6936 - val_loss: 0.0621 - val_acc: 0.6667 - val_f1_m: 0.7488
Epoch 4/5
437/437 [==============================] - 422s 966ms/step - loss: 0.0680 - acc: 0.6570 - f1_m: 0.7210 - val_loss: 0.0568 - val_acc: 0.6943 - val_f1_m: 0.7685
Epoch 5/5
437/437 [==============================] - 422s 967ms/step - loss: 0.0638 - acc: 0.6703 - f1_m: 0.7416 - val_loss: 0.0523 - val_acc: 0.7029 - val_f1_m: 0.7941
sleeping


In [14]:

epochs = 2
batch_size = 128
val_data = BertSemanticDataGenerator(
    test_X['ABSTRACT'].values.astype("str"),
    test_X.iloc[:,1:6].values.astype("int32"),
    test_y,
    batch_size=batch_size,
    shuffle=False,
)


data = BertSemanticDataGenerator(
    X['ABSTRACT'].values.astype("str"),
    X.iloc[:,1:6].values.astype("int32"),
    y,
    batch_size=batch_size,
    shuffle=True,
)

print("training trainable bert model")

history = model.fit(
    data,
    validation_data=val_data,
    epochs=epochs,
)


val_data = BertSemanticDataGenerator(
    test_X['ABSTRACT'].values.astype("str"),
    test_X.iloc[:,1:6].values.astype("int32"),
    test_y,
    batch_size=1,
    shuffle=False,
)


print("predicting validation data")
preds = model.predict_generator(val_data, verbose=1, use_multiprocessing=True)




print("evaluating validation data")
pkl.dump(test_y, open("val_original.pkl", "wb"))
pkl.dump(preds, open("val_preds.pkl", "wb"))
model.evaluate(val_data, verbose=1)


tdf = pd.read_csv("/kaggle/input/hacklive-3-guided-hackathon-nlp/Test.csv")


val_data = BertSemanticDataGenerator(
    tdf['ABSTRACT'].values.astype("str"),
    tdf.iloc[:,2:6].values.astype("int32"),
    batch_size=2,
    labels=None,
    shuffle=False,
    include_targets=False
)


print("predicting test data")
preds = model.predict_generator(val_data, verbose=1, use_multiprocessing=True)


print("final predictions")
print(preds)

pkl.dump(preds, open('pred_proba.pkl', 'wb'))



training trainable bert model
Epoch 1/2
109/109 [==============================] - 394s 4s/step - loss: 0.0581 - acc: 0.6907 - f1_m: 0.7683 - val_loss: 0.0484 - val_acc: 0.7113 - val_f1_m: 0.8085
Epoch 2/2
109/109 [==============================] - 393s 4s/step - loss: 0.0558 - acc: 0.6990 - f1_m: 0.7795 - val_loss: 0.0470 - val_acc: 0.7210 - val_f1_m: 0.8105
predicting validation data
2801/2801 [==============================] - 188s 67ms/step
evaluating validation data
2801/2801 [==============================] - 182s 65ms/step - loss: 0.0466 - acc: 0.7222 - f1_m: 0.8090
predicting test data
3001/3001 [==============================] - 274s 91ms/step
final predictions
[[4.7558631e-05 1.1307270e-02 9.6181147e-02 ... 4.7212100e-04
  1.6809901e-04 2.0960410e-04]
 [1.4751501e-04 7.1922654e-01 2.5488198e-02 ... 4.9949839e-04
  4.0903818e-04 1.7602278e-02]
 [3.0159997e-02 2.8757462e-03 3.7754685e-04 ... 1.3196630e-03
  2.0742104e-03 2.1739595e-03]
 ...
 [2.5280016e-05 2.0353671e-03 1.23291

In [15]:


pred = 'val_preds.pkl'
y = 'val_original.pkl'
final_pred = 'pred_proba.pkl'



In [16]:

pred = pkl.load(open(pred, 'rb'))

y = pkl.load(open(y, 'rb'))
y = y[0:0 + pred.shape[0]]

thresholds = [0] * 25
counts = [0] * 25
y_trues = [0] * 25

def get_current_loss(p_col, true_col, th):
    p_col = sum(p_col > th) / len(p_col)
    true_col = sum(true_col) / len(true_col)
    return abs(p_col - true_col)


for col in range(25):
    current_loss = sys.maxsize
    temp_loss = sys.maxsize
    for threshold in np.linspace(0, 1, 40, endpoint=False):
        pred_col = np.take(pred, col, axis=1)
        y_col = np.take(y, col, axis=1)
        temp_loss = get_current_loss(pred_col, y_col, threshold)
        if  temp_loss < current_loss:
            thresholds[col] = threshold
            current_loss = temp_loss
            counts[col] = sum(pred_col > threshold) / len(pred_col)
            y_trues[col] = sum(y_col) / len(y_col)
            # print(col, current_loss, thresholds[col], counts[col], y_trues[col])


print(thresholds)
print(counts)
print(y_trues)


[0.3375, 0.28750000000000003, 0.30000000000000004, 0.6125, 0.48750000000000004, 0.3125, 0.47500000000000003, 0.2625, 0.23750000000000002, 0.36250000000000004, 0.28750000000000003, 0.275, 0.3125, 0.4375, 0.4375, 0.4, 0.28750000000000003, 0.325, 0.4, 0.5125000000000001, 0.41250000000000003, 0.28750000000000003, 0.4, 0.4625, 0.4]
[0.04248482684755445, 0.04141378079257408, 0.09532309889325241, 0.03641556586933238, 0.04641199571581578, 0.07283113173866476, 0.047483041770796146, 0.03248839700107105, 0.03855765797929311, 0.035344519814352014, 0.027490182077829346, 0.026776151374509102, 0.034987504462691894, 0.26347732952516956, 0.0567654409139593, 0.04212781149589433, 0.026062120671188863, 0.04891110317743663, 0.02034987504462692, 0.07640128525526597, 0.043198857550874686, 0.0367725812209925, 0.0649767940021421, 0.03784362727597287, 0.0467690110674759]
[0.04248482684755445, 0.04212781149589433, 0.09460906818993217, 0.03641556586933238, 0.04641199571581578, 0.07247411638700464, 0.0481970724741

In [17]:

df = pd.read_csv('/kaggle/input/hacklive-3-guided-hackathon-nlp/Train.csv')

y = df.iloc[:, 6: 6 + 25]
LABELS = y.columns


tdf = pd.read_csv("/kaggle/input/hacklive-3-guided-hackathon-nlp/Test.csv")
i = pkl.load(open(final_pred, "rb"))

for col in range(len(LABELS)):
    tdf[LABELS[col]] = [1 if x[col] > thresholds[col] else 0 for x in i]
    

tdf.drop(columns=['ABSTRACT', 'Computer Science', 'Mathematics', 'Physics', 'Statistics']).to_csv("final1.csv", index=False)

In [18]:
from IPython.display import FileLink
filename = "final.csv"
FileLink(filename)

/kaggle/working/final.csv

In [19]:

pred = 'val_preds.pkl'
y = 'val_original.pkl'
final_pred = 'pred_proba.pkl'

In [20]:
FileLink(pred)

/kaggle/working/val_preds.pkl

In [21]:
FileLink(y)

/kaggle/working/val_original.pkl

In [22]:
FileLink(final_pred)

/kaggle/working/pred_proba.pkl